In [1]:
import os
os.chdir('/content/drive/MyDrive/MVA/KKML')

# Kernel Methods: Challenge

Julia Linhart, Roman Castagné, Louis Bouvier

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial
from scipy.spatial import distance_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV
import cvxpy as cp
import warnings
import time
from itertools import product
from numba import jit

from utils import run_model, write_csv


warnings.filterwarnings("ignore", category=DeprecationWarning)

# I) Preprocessing

In [2]:
data_folder = 'data' # 'machine-learning-with-kernel-methods-2021'

X_train_1 = pd.read_csv(f'{data_folder}/Xtr2_mat100.csv', sep = ' ', index_col=False, header=None)
y_train_1 = pd.read_csv(f'{data_folder}/Ytr2.csv')

In [4]:
y_train_1.describe()

,Id,Bound
count,2000.000000,2000.000000
mean,4999.500000,0.498500
std,577.494589,0.500123
min,4000.000000,0.000000
25%,4499.750000,0.000000
50%,4999.500000,0.000000
75%,5499.250000,1.000000
max,5999.000000,1.000000


In [5]:
y_train_1 = np.array(y_train_1)[:,1]

In [6]:
X_train_1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.010565,0.010201,0.010375,0.011587,0.011609,0.010707,0.009359,0.011957,0.009571,0.010582,0.009424,0.009793,0.012848,0.012092,0.011196,0.010364,0.009875,0.010962,0.010185,0.008342,0.010734,0.010038,0.011554,0.008995,0.010283,0.008647,0.008886,0.008826,0.007821,0.009761,0.008533,0.011864,0.009299,0.010641,0.009560,0.008929,0.010217,0.009641,0.009880,0.010038,...,0.009511,0.010614,0.011957,0.009641,0.011772,0.009500,0.008783,0.010005,0.010870,0.009147,0.013565,0.010587,0.009793,0.010908,0.009500,0.009772,0.009103,0.010147,0.008587,0.010538,0.010897,0.008913,0.008630,0.008380,0.009016,0.011478,0.008832,0.009989,0.010587,0.008625,0.007951,0.009457,0.008554,0.009283,0.008261,0.009614,0.011141,0.009777,0.008217,0.008565
std,0.012278,0.010723,0.011467,0.011453,0.012182,0.010478,0.009789,0.012444,0.013805,0.013652,0.012934,0.011163,0.027178,0.018160,0.011200,0.010356,0.010089,0.019951,0.010631,0.009920,0.011238,0.010962,0.011475,0.009723,0.010922,0.009933,0.009622,0.009861,0.010099,0.010628,0.009945,0.010829,0.010358,0.010460,0.011039,0.009612,0.010705,0.012258,0.020208,0.011266,...,0.010436,0.011172,0.012915,0.010912,0.011305,0.016977,0.014644,0.012108,0.011800,0.009647,0.011868,0.011752,0.013102,0.010237,0.009652,0.009687,0.011871,0.010457,0.012348,0.011010,0.011005,0.010695,0.009248,0.010494,0.009279,0.011204,0.010571,0.015973,0.009745,0.011904,0.009605,0.009701,0.009350,0.009741,0.012341,0.010338,0.010863,0.010402,0.009709,0.009283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [ ]:
X_train_1 = np.array(X_train_1)
X_train_1 = (X_train_1 - X_train_1.mean(axis=0))/X_train_1.std(axis=0)

# II) First linear models of the mat100 input

## A) Logistic regression

In [22]:
from utils import run_model

run_model('logreg')

C:\Users\roman\Documents\Academic\2020_MVA\S2_KernelMethods\KM_challenge\functions.py:103: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))
C:\Users\roman\Documents\Academic\2020_MVA\S2_KernelMethods\KM_challenge\functions.py:111: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))


Accuracy on train set 0: 0.62
Accuracy on test set 0 : 0.56
Accuracy on train set 1: 0.60
Accuracy on test set 1 : 0.59
Accuracy on train set 2: 0.70
Accuracy on test set 2 : 0.66


In [ ]:
from utils import write_csv

ids = np.arange(all_y_eval.shape[0])
filename = "results/submission_log_reg.csv"

# write_csv(ids, all_y_eval, filename)

## B) Ridge regression

In [10]:
run_model('rr')

Accuracy on train set 0: 0.65
Accuracy on test set 0 : 0.60
Accuracy on train set 1: 0.64
Accuracy on test set 1 : 0.57
Accuracy on train set 2: 0.73
Accuracy on test set 2 : 0.69


# III) Kernel baselines 

## A) Gaussian Kernel

### a) Kernel Ridge Regression

In [5]:
## run kernel ridge regression with gaussian kernel
run_model('krr', kernel='gaussian', prop_test=0.2, use_grid_search=True)

{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5789473684210527}
Accuracy on train set 0: 1.00
Accuracy on test set 0 : 0.57
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.6578947368421053}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.59
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5}
Accuracy on train set 2: 1.00
Accuracy on test set 2 : 0.67


### b) Kernel SVM

In [9]:
## run kernel SVM with gaussian kernel
run_model('ksvm', kernel='gaussian', prop_test=0.2)

{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 0: 0.99
Accuracy on test set 0 : 0.62
{'kernel': 'gaussian', 'lamb': 1e-10, 'sigma': 1.0}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.60
{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 2: 0.99
Accuracy on test set 2 : 0.72


## B) Spectrum kernel

In [3]:
from kernels import Spectrum_kernel

In [38]:
# Example when using a precomputed kernel
K = []
for name in [0, 1, 2]:
    X    = np.array(pd.read_csv(f'{data_folder}/Xtr{name}.csv')['seq'])
    X_ev = np.array(pd.read_csv(f'{data_folder}/Xte{name}.csv')['seq'])
    
    t0 = time.time()
    K_tr = Spectrum_kernel(X, X, k=6)
    print(f"Time to compute train kernel : {time.time() - t0}")
    K_te = Spectrum_kernel(X, X_ev, k=6)
    
    K.append({"train": K_tr, "eval": K_te})

Time to compute train kernel : 0.7150859832763672
Time to compute train kernel : 0.6163516044616699
Time to compute train kernel : 0.6303155422210693


In [39]:
## run kernel SVM with gaussian kernel
run_model('ksvm', kernel='spectrum', K=K, sequence=True, prop_test=0.2)

KeyboardInterrupt: 

### a) Kernel SVM


## C) Substring kernel

In [13]:
from kernels import substring_similarity, substring_kernel

In [12]:
# Run similarity between two strings
t0 = time.time()
# k = substring_similarity("ATGCATGATGCATG", "ATGCATCATGATGT", 3, 1.)
# k = substring_similarity("ATGC", "ATGC", 3, 0.7)
k = substring_similarity("cat", "cat", 1, 0.7)
k_expected = 2 * 0.7 ** 4 + 0.7 ** 6
print(f"Time to compute : {time.time() - t0:.4f}s")
print(f"Value : {k}")
print(f"Expected value : {k_expected}")

Time to compute : 2.0617s
Value : 0.9799999999999999
Expected value : 0.5978489999999999


In [14]:
# Run kernel computation between N strings
X = pd.read_csv(f'{data_folder}/Xtr0.csv', sep = ',').to_numpy()
X = X[:100,1]
t0 = time.time()
K = substring_kernel(X, X, k=5, lambd=0.7)
print(f"Time to compute K : {time.time() - t0:.2f}s")
# print(K)

Time to compute K : 76.18s


## D) Mismatch kernel

In [2]:
from kernels import spectrum

In [3]:
alphabet = ["A", "T", "G", "C"]

In [13]:
class Node(object):
    def __init__(self, parent, letter):
        self.parent = parent
        self.letter = letter
        self.sequence = None
        self.pointers = {}
        
    def get_sequence(self):
        return self.sequence
    
    def get_parent(self):
        return self.parent
    
    def get_pointers(self):
        return self.pointers
    
    def set_sequence(self):
        if self.parent:
            self.sequence = self.parent.get_sequence()+self.letter if self.parent.get_sequence() else self.letter
    
    def set_pointers(self, dataset, depth, max_mismatch):
        Pointers = {}
        if self.get_parent() is not None:
            parent_Pointers = self.get_parent().get_pointers()
            for pointer, mismatch in zip(parent_Pointers.keys(), parent_Pointers.values()): 
                if dataset[pointer][depth]!=self.letter:
                    new_mismatch = mismatch+1
                    if new_mismatch <= max_mismatch:
                        Pointers[pointer] = new_mismatch
                else:
                    Pointers[pointer] = mismatch
        else:
            for i in range(len(dataset)):
                Pointers[i] = 0
        self.pointers = Pointers

In [17]:
class Tree(Node):
    def __init__(self,k,m,dataset):
        self.maxdepth = k
        self.max_nb_mismatches = m
        # create the strings of length k with the alphabet
        self.dataset = dataset
        # create the root node with no letter and no parent
        root = Node(parent=None, letter=None)
        root.set_pointers(self.dataset, 0, self.max_nb_mismatches)
        # create a dictionnary of nodes: width first
        self.Nodes = {'0': [root]}
        for d in range(1,self.maxdepth+1):
            self.Nodes[str(d)] = []
        count=0
        while count<self.maxdepth:
            for parent_ in self.Nodes[str(count)]:
                for charact in alphabet:
                    child = Node(parent_,charact)
                    child.set_pointers(self.dataset, count, self.max_nb_mismatches)
                    self.Nodes[str(count+1)].append(child)
            count+=1                
            
    def get_Nodes(self):
        return self.Nodes
    
    def build_kernel(self):
        nb_substrings_per_string = 101-self.maxdepth+1
        samples = self.dataset.reshape(-1,nb_substrings_per_string)
        K = np.zeros((samples.shape[0], samples.shape[0]))
        for leaf in self.Nodes[str(self.maxdepth)]:
            one_hot = np.zeros(len(self.dataset))
            one_hot[list(leaf.get_pointers().keys())]=1
            occurences = one_hot.reshape(-1,nb_substrings_per_string).sum(axis=1)
            print(occurences)
            K = K + occurences.T@occurences
        return K

In [18]:
def Mismatch_kernel(X1, X2, k, m):
    """inputs:
    - X1 (size N1xd): a set of strings
    - X2 (size N2xd): another one
    - k (integer): the length of the substrings considered
    - m (integer): the order of mismatch accepted
    ouput:
    - the associated (N1+N2)x(N1+N2) mismatch kernel
    """
    aggregated_data = np.hstack((X1,X2))
    dataset_k = np.array([spectrum(x,k) for x in aggregated_data])
    dataset_k = dataset_k.reshape(-1)
    Test_tree = Tree(k=k, m=m, dataset=dataset_k)
    kernel = Test_tree.build_kernel()
    return kernel

In [10]:
k = 4
m = 1
Nb_samples = 2000
prop_test = 0.1

X = pd.read_csv(f'{data_folder}/Xtr0.csv', sep = ',').to_numpy()[:,1]
y = pd.read_csv(f'{data_folder}/ytr0.csv', sep = ',').to_numpy()[:,1]

tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

X_tr = X[tr_indices]
X_te = X[te_indices]

y_tr = y[tr_indices]
y_te = y[te_indices]

In [158]:
K_m = Mismatch_kernel(X_tr,X_te,k,m)

In [19]:
X_tr = X_tr[:1]
X_te = X_te[:1]

K_m = Mismatch_kernel(X_tr,X_te,k,m)

[10.  5.]
[ 6. 11.]
[8. 4.]
[9. 3.]
[7. 4.]
[5. 9.]
[3. 4.]
[3. 8.]
[11.  7.]
[6. 4.]
[5. 4.]
[5. 4.]
[13.  7.]
[9. 8.]
[5. 2.]
[7. 5.]
[6. 8.]
[ 3. 12.]
[7. 5.]
[5. 6.]
[5. 8.]
[ 3. 14.]
[3. 6.]
[1. 4.]
[3. 3.]
[ 7. 11.]
[3. 1.]
[4. 2.]
[10.  7.]
[6. 9.]
[1. 5.]
[1. 9.]
[11.  4.]
[2. 7.]
[8. 6.]
[3. 3.]
[10.  6.]
[5. 9.]
[2. 4.]
[5. 4.]
[6. 1.]
[5. 7.]
[5. 1.]
[4. 0.]
[10.  5.]
[8. 7.]
[4. 1.]
[5. 3.]
[13.  7.]
[9. 5.]
[8. 4.]
[9. 4.]
[7. 7.]
[4. 7.]
[11.  2.]
[3. 3.]
[4. 2.]
[5. 5.]
[4. 0.]
[2. 1.]
[11.  7.]
[ 5. 10.]
[6. 4.]
[3. 6.]
[ 8. 12.]
[4. 4.]
[6. 5.]
[6. 8.]
[ 3. 11.]
[ 2. 15.]
[3. 8.]
[7. 8.]
[5. 6.]
[5. 8.]
[5. 3.]
[6. 6.]
[6. 6.]
[6. 6.]
[2. 3.]
[ 9. 11.]
[3. 7.]
[ 3. 15.]
[3. 8.]
[7. 7.]
[ 2. 15.]
[ 4. 14.]
[6. 8.]
[ 3. 17.]
[3. 5.]
[ 6. 12.]
[5. 4.]
[5. 5.]
[5. 7.]
[ 5. 14.]
[2. 5.]
[2. 8.]
[3. 5.]
[3. 7.]
[2. 2.]
[8. 2.]
[6. 8.]
[ 5. 11.]
[10.  7.]
[5. 8.]
[3. 3.]
[7. 3.]
[4. 1.]
[7. 2.]
[7. 3.]
[6. 5.]
[4. 2.]
[4. 5.]
[5. 5.]
[2. 7.]
[9. 4.]
[6. 7.]
[ 5. 10.]
[ 7. 12.

In [159]:
print(np.amin(np.linalg.eig(K_m)[0].real))

-2.9075945852892406e-06


In [160]:
K_m = K_m + np.eye(K_m.shape[0])*pow(10,-4)

In [161]:
K_train = K_m[:1800,:1800]
K_pred = K_m[:1800,1800:]

In [162]:
N_tr = K_train.shape[0]
lamb = 1
# Define QP and solve it with cvxpy
alpha = cp.Variable(N_tr)
objective = cp.Maximize(2*alpha.T@y_tr - cp.quad_form(alpha, K_train))
constraints = [0 <= cp.multiply(y_tr,alpha), cp.multiply(y_tr,alpha) <= 1/(2*lamb*N_tr)]
prob = cp.Problem(objective, constraints)
# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
alpha_ = alpha.value

In [163]:
y_pred_te = 2 * (alpha_.T@K_pred > 0).reshape(-1, ).astype("int") - 1
y_pred_tr = 2 * (alpha_.T@K_train > 0).reshape(-1, ).astype("int") - 1

In [164]:
print(np.sum(y_pred_tr == y_tr)/y_tr.shape[0])
print(np.sum(y_pred_te == y_te)/y_te.shape[0])

0.4777777777777778
0.51
